In [ ]:
from ultralytics import YOLO
from yolo_config import Config
import os
import pandas as pd
import shutil

In [2]:
for model_size in Config.MODEL_SIZES:
    
    model_dir = os.path.join(Config.RUNS_DIR, Config.DATASET, model_size)
    metrics_by_fold = []
    
    for fold in range(1,11):

        # Dataset

        fold_dir = f'fold_{fold}'
        yaml_path = os.path.join(Config.DATASET, fold_dir, Config.YAML_FILE)
        project = os.path.join(model_dir, fold_dir)


        # Train

        model = YOLO(f"./Weights/{model_size}")
        model.train(
            data = yaml_path,
            imgsz = Config.IMG_SIZE,
            # epochs = Config.EPOCHS,
            epochs = 1,
            batch = Config.BATCH_SIZE,
            project = project,
        )
        
        # Validate
        
        weights_dir = os.path.join(project, 'train', 'weights')
        
        best_weights = os.path.join(weights_dir, 'best.pt')
        
        model = YOLO(best_weights)
        
        val_metrics = model.val(
            data = yaml_path, 
            split = 'val', 
            project = project, 
        )
        
        confusion_matrix_keys = [f'CM({i},{j})' for i in range(3) for j in range(3)]
        confusion_matrix_values =  val_metrics.confusion_matrix.matrix.flatten()
        confusion_matrix_dict = {k:v for (k,v) in zip(confusion_matrix_keys, confusion_matrix_values)}

        metrics = val_metrics.results_dict
        metrics.update(confusion_matrix_dict)
        
        metrics_by_fold.append(metrics)
        
        shutil.rmtree(weights_dir)
        
    
        break # Fold
    
    # Aggregate Metrics
    
    index = [f'fold {fold}' for fold in range(1,11)]
    metrics_by_fold = pd.DataFrame(metrics_by_fold, index=index)
    metrics_by_fold.loc['mean'] = metrics_by_fold.mean()
    
    metrics_path = os.path.join(model_dir, 'metrics.csv')
    metrics_by_fold.to_csv(metrics_path)
    
    break # Model Size


New https://pypi.org/project/ultralytics/8.0.181 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.164  Python-3.8.17 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=detect, mode=train, model=./Weights/yolov8n.pt, data=InBreast_Yolov8\fold_1\data.yaml, epochs=5, patience=50, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs_usfq_server\InBreast_Yolov8\yolov8n.pt\fold_1, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride

: 